# Third-level GLM using Nipype FSL

Notebook compiled by Yibei Chen

In this notebook, we recreate the whole preprocess of FSL GUI using nipype code. For each nipype node, we list the corresponding fsl command from the log file. The dataset we use is a Flanker task, which can be downloaded [here](https://openneuro.org/datasets/ds000102/versions/00001).

We also borrow some helps from this [document](https://nipype.readthedocs.io/en/latest/users/examples/fmri_fsl.html). 

## Preparation
Import all the relevant libraries needed for the preprocessing stage.

In [1]:
from __future__ import print_function
from __future__ import division
from builtins import str
from builtins import range

import os, stat  # system functions
import getpass
from glob import glob

import numpy as np
from nipype import Function
import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.fsl as fsl  # fsl
import nipype.interfaces.utility as util  # utility
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.algorithms.modelgen as model  # model generation
import nipype.algorithms.rapidart as ra  # artifact detection

%autosave 5

Autosaving every 5 seconds


Set up data path

In [2]:
# Get current user
user = getpass.getuser()
print('Running code as: ', user)

# Input: Set the second-level result path
data_dir = '/home/{}/out/fsl/hw2/level2_results/'.format(user)
# Output: Set path where nipype will store stepwise results
exp_dir = '/home/{}/out/fsl/hw2/'.format(user)

try:
    os.mkdir(exp_dir)
except OSError as error:
    print(error)
    
    
# Grant root write access to our output files 
os.chmod(exp_dir, os.stat(exp_dir).st_mode | ((stat.S_IRWXU | stat.S_IRWXO)))

Running code as:  yc
[Errno 17] File exists: '/home/yc/out/fsl/hw2/'


Start the workflow

In [3]:
wf = pe.Workflow(name='level3', base_dir=exp_dir)
wf.config["execution"]["crashfile_format"] = "txt"

The following two nodes (`infosource` & `dg`) together define all inputs required for the preprocessing workflow

In [4]:
# we want to group the outcome by contrast not subject
contr_list = [1,2,3,4]
infosource = pe.Node(util.IdentityInterface(fields=["contr_id"]),
                  name="infosource")
infosource.iterables = [("contr_id", contr_list)]

In [5]:
# here we use SelectFiles, instead of DataGrabber, because the former is more flexible with formatting syntax
templates = {
    "copes":"contrast_{contr_id}/*/_contr_id_{contr_id}/*/cope*.nii.gz",
    "varcopes":"contrast_{contr_id}/*/_contr_id_{contr_id}/*/varcope*.nii.gz",
    "masks":"contrast_{contr_id}/*/_contr_id_{contr_id}/*/mask.nii.gz"
}
dg = pe.Node(interface=nio.SelectFiles(templates),
             name="selectfiles")
dg.inputs.base_directory = data_dir

wf.connect([
        (infosource, dg, [("contr_id", "contr_id")])
])

## Third-level GLM

Combining results from multiple runs of one subject into one

### Higher-level input files preparation

#### Step 1: Merge registered copes & varcopes & masks


**Corresponding FSL command:**
```
/usr/local/fsl/bin/fslmerge -t mask (masks from all 26 inputs)
/usr/local/fsl/bin/fslmerge -t cope (copes from all 26 inputs)
/usr/local/fsl/bin/fslmerge -t varcop (varcopes from all 26 inputs)
```

In [6]:
copemerge = pe.Node(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.Node(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

maskmerge = pe.Node(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="maskmerge")


def repeat_mask(file):
    n_sub = 26
    import numpy as np
    mask_lst = [file]
    repeated = np.repeat(mask_lst,26)
    return list(repeated)


wf.connect(dg, 'copes', copemerge, 'in_files')
wf.connect(dg, 'varcopes', varcopemerge, 'in_files')
wf.connect(dg, ('masks',repeat_mask), maskmerge, 'in_files')

#### Step 2: Making mask

In FSL, there are many commands about `maskunique`, which is unless for the second level. We can ignore it.

**Corresponding FSL command:**
```
/usr/local/fsl/bin/fslmaths mask -Tmin mask
```

In [7]:
# /usr/local/fsl/bin/fslmaths mask -Tmin mask
minmask = pe.Node(
    interface=fsl.ImageMaths(op_string='-Tmin'),
    iterfield=['in_file'],
    name='minmask')

wf.connect(maskmerge, 'merged_file', minmask, 'in_file')


#### Step 3: Masking copes & varcopes

**Corresponding FSL command:**

we have four contrasts so the following commands repeat four times

```
/usr/local/fsl/bin/fslmaths cope1 -mas mask cope1
/usr/local/fsl/bin/fslmaths varcope1 -mas mask varcope1
```

In [8]:
maskcope = pe.Node(
    interface=fsl.ImageMaths(op_string='-mas'),
    iterfield=['in_file', 'in_file2'],
    name='maskcope')

maskvarcope = pe.Node(
    interface=fsl.ImageMaths(op_string='-mas'),
    iterfield=['in_file', 'in_file2'],
    name='maskvarcope')

wf.connect(copemerge, 'merged_file', maskcope, 'in_file')
wf.connect(minmask, 'out_file', maskcope, 'in_file2')
wf.connect(varcopemerge, 'merged_file', maskvarcope, 'in_file')
wf.connect(minmask, 'out_file', maskvarcope, 'in_file2')

### Set up second-level contrasts and mixed-effects

Since we only have a single-group set-up, we can actually use [L2Model](https://nipype.readthedocs.io/en/latest/api/generated/nipype.interfaces.fsl.model.html#l2model).
If we have an ANOVA-like design, using [MultipleRegressDesign](https://nipype.readthedocs.io/en/latest/api/generated/nipype.interfaces.fsl.model.html#multipleregressdesign) would be a better option.

In [9]:
def num_copes(files):
    return len(files)

level3model = pe.Node(interface=fsl.L2Model(), name='l3model')
wf.connect(dg, ('copes',num_copes), level3model, 'num_copes')

In [10]:
level3estimate = pe.Node(
    interface=fsl.FLAMEO(run_mode='flame1'),
    name="level3estimate",
    iterfield=['cope_file', 'var_cope_file'])

wf.connect([
    (maskcope, level3estimate, [('out_file', 'cope_file')]),
    (maskvarcope, level3estimate, [('out_file', 'var_cope_file')]),
    (minmask, level3estimate, [('out_file', 'mask_file')]),
    (level3model, level3estimate, [('design_mat', 'design_file'),
                                   ('design_con', 't_con_file'), 
                                   ('design_grp', 'cov_split_file')]),
])

In [11]:
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=os.path.join(exp_dir, "level3_results")

int2string = lambda x: 'contrast_'+str(x)
    
wf.connect(infosource, ('contr_id',int2string), datasink, 'container')
wf.connect([(level3estimate, datasink, [('stats_dir', 'stats_dir')])])

In [13]:
# Run Workflow
wf.run(plugin="MultiProc", plugin_args={"n_procs": 8})

211209-04:15:34,148 nipype.workflow INFO:
	 Workflow level3 settings: ['check', 'execution', 'logging', 'monitoring']
211209-04:15:34,185 nipype.workflow INFO:
	 Running in parallel.
211209-04:15:34,187 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 27.96/27.96, Free processors: 8/8.
211209-04:15:34,260 nipype.workflow INFO:
	 [Node] Setting-up "level3.selectfiles" in "/home/yc/out/fsl/hw2/level3/_contr_id_4/selectfiles".
211209-04:15:34,262 nipype.workflow INFO:
	 [Node] Setting-up "level3.selectfiles" in "/home/yc/out/fsl/hw2/level3/_contr_id_3/selectfiles".
211209-04:15:34,264 nipype.workflow INFO:
	 [Node] Setting-up "level3.selectfiles" in "/home/yc/out/fsl/hw2/level3/_contr_id_2/selectfiles".
211209-04:15:34,267 nipype.workflow INFO:
	 [Node] Setting-up "level3.selectfiles" in "/home/yc/out/fsl/hw2/level3/_contr_id_1/selectfiles".
211209-04:15:34,288 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")